In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output, display

In [ ]:


URL = "https://results.baa.org/{}/?page={}&event=R&event_main_group=runner&pid=list&search%5Bsex%5D={}&search%5Bage_class%5D={}&num_results=1000"
YEAR= 2023
NAME = "boston"

SEX = ['M', 'W', 'X'] 
AGE = {
    "1":"18-39","2":"40-44","9":"45-49","3":"50-54","10":"55-59","4":"60-64","11":"65-69","8":"70-74","12":"75-79","13":"80+"
}


In [ ]:
df_final = None

for sex in SEX:
    for (key, age) in AGE.items():
            
        page_number = 1

        while True:
            clear_output()
            display(f"Retrieving page {page_number} for {sex} {age}")

            res = requests.get(URL.format(YEAR, page_number, sex, key))
            page = BeautifulSoup(res.text)

            rows = page.select("li.list-group-item")


            if len(rows) == 3 and rows[-1].select("div.alert-info"):
                break

            # get time
            rows2 = [r.select("div.type-time")[-2].find(string=True, recursive=False) for r in rows[2:]]

            df = pd.DataFrame(rows2, columns=['time'])
            df.insert(0, "Age", age)
            df.insert(0, "Sex", sex)

            if df_final is None:
                df_final = df
            else:
                df_final = pd.concat([df_final, df])

            page_number += 1

df_final.shape

In [ ]:
df_final.head(4)

In [ ]:

df_final.to_csv(f"{NAME}_{YEAR}.csv")

In [ ]:
df_final.pivot_table(index='Age', columns='Sex', aggfunc='count', fill_value=0)